In [2]:
!pip install scikit-learn



Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 995.6 kB/s eta 0:00:09
   --- ------------------------------------ 0.8/8.7 MB 1.1 MB/s eta 0:00:08
   ---- ----------------------------------- 1.0/8.7 MB 1.2 MB/s eta 0:00:07
   ------ --------------------------------- 1.3/8.7 MB 1.2 MB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.7 MB 1.1 MB/s eta 0:00:07
   -------- ------------------------------- 1.8/8.7 MB 1.1 MB/s eta 0:00:07
   -------- ------------------------------- 1.8/8.7 MB 1.1 MB/s eta 0:00:07
   -------- ------------------------------- 1.8/8.7 MB 1.1 MB/s eta 0:00:07
   --------- ------------------------------ 2.1/8.7 MB 959.4 kB/s eta 0:00:07
   --------- --------------

In [ ]:
import sklearn 
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report


In [11]:
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('C:/Users/hp/Desktop/SIDATA M2/Fouille de données/training.1600000.processed.noemoticon.csv', encoding='latin-1',engine="python",on_bad_lines="skip", header=None , sep=',')
df = df[[0, 5]]#garder uniquement la colonne sentiment et text
df.columns = ['polarity', 'text']
print(df.head())

   polarity                                               text
0         0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1         0  is upset that he can't update his Facebook by ...
2         0  @Kenichan I dived many times for the ball. Man...
3         0    my whole body feels itchy and like its on fire 
4         0  @nationwideclass no, it's not behaving at all....


In [6]:
#modification des polarités
df = df[df.polarity != 2]#garder que les polarités différent de 2

df['polarity'] = df['polarity'].map({0: 0, 4: 1})

print(df['polarity'].value_counts())

polarity
0    800000
1    800000
Name: count, dtype: int64


In [7]:
#nettoyage du texte 
def clean_tweet(text):
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = text.lower()
    text = re.sub(r'[^a-z0-9#\s]', ' ', text)
    return text


In [8]:
#appliquer la fonction précédente
df['clean_text'] = df['text'].apply(clean_tweet)

In [9]:

#vérifier les changements 
df.head()

,polarity,text,clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got da...
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to s...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why am...


In [10]:
tfidf = TfidfVectorizer(
    max_features=5000,       # nombre de mots caractéristiques
    ngram_range=(1,2),       # uni-grams et bi-grams
    lowercase=False,         # déjà mis en minuscules
    analyzer='word'
)
X = tfidf.fit_transform(df['clean_text'])


In [12]:
Y = (df['polarity'] == 1).astype(int)  # 1 = positif, 0 = négatif

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [13]:
#modéle 1
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

bnb_pred = bnb.predict(X_test)

print("Bernoulli Naive Bayes Accuracy:", accuracy_score(y_test, bnb_pred))
print("\nBernoulliNB Classification Report:\n", classification_report(y_test, bnb_pred))

Bernoulli Naive Bayes Accuracy: 0.767234375

BernoulliNB Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.75      0.76    159494
           1       0.76      0.79      0.77    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [14]:
#modéle 2
svm = LinearSVC(max_iter=1000)
svm.fit(X_train, y_train)

svm_pred = svm.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("\nSVM Classification Report:\n", classification_report(y_test, svm_pred))

SVM Accuracy: 0.795425

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



In [15]:
#modele3
logreg = LogisticRegression(max_iter=100)
logreg.fit(X_train, y_train)

logreg_pred = logreg.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, logreg_pred))
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, logreg_pred))

Logistic Regression Accuracy: 0.7956625

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000

